In [1]:
import sys
import pandas

sys.path.append(".")
import processamento

In [2]:
quantidade_lida = 1000
enem_2019 = pandas.read_csv(
    "./microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv",
    delimiter=";",
    encoding="iso-8859-15",
    nrows=quantidade_lida,
)
enem_2020 = pandas.read_csv(
    "./microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv",
    delimiter=";",
    encoding="iso-8859-15",
    nrows=quantidade_lida,
)
enem_2019, enem_2020 = processamento.limpeza(enem_2019, enem_2020)

calcular a correlação das variaveis, cramers é a unica da biblioteca que pode ser usada para variaveis categoricas não ordinais

In [3]:
from pandas_profiling import ProfileReport
def profile(enem,ano):
    import json
    #usar apenas a correlação de cramer pq é a unica que funcionar para valores nominais
    profile = enem.profile_report(
        show_variable_description=False,
        progress_bar=False,
        correlations={
            "pearson": {"calculate": False},
            "spearman": {"calculate": False},
            "kendall": {"calculate": False},
            "phi_k": {"calculate": False},
            "cramers": {"calculate": True},})
    profile.to_file(f"enem-{ano}.html")
    return json.loads(profile.to_json())

correlacao_2019 = profile(enem_2019,"2019")["correlations"]["cramers"]
correlacao_2020 = profile(enem_2020,"2020")["correlations"]["cramers"]

pega o calculo de correlação gerado pela biblioteca ProfileReport,a biblioteca faz seu proprio arquivo html,
que não é possivel filtrar quais campos serão incluidas nele, essa função filtra os campos e as variaveis e 
faz o grafico

In [ ]:
def gera_tabela_correlacao(csv,ano):
    correlacao = []
    for i in range(len(csv)):
        correlacao.append(dict(csv[i]))

    correlacao = pandas.DataFrame(correlacao)

    correlacao = pandas.concat([correlacao,pandas.DataFrame(correlacao.columns)],axis=1)
    correlacao = correlacao.rename(columns ={0:"colunas"})
    
    correlacao = correlacao.set_index('colunas').sort_values(by="colunas")
    correlacao = correlacao[sorted(correlacao.columns)]
    correlacao.to_csv(f"correlacao_{ano}.csv")
    correlacao.style\
        .background_gradient(cmap='coolwarm')\
        .format(precision=2)\
        .to_html(f"correlacao_{ano}.html")

gera_tabela_correlacao(correlacao_2019,"2019")
gera_tabela_correlacao(correlacao_2019,"2020")    